# 第13章 管理サイト(Django Admin)

## 13.1 概要

### 管理サイト（Django Admin）
- 開発時にちょっとした検証をしたいときや、テスト用のデータをデータベースに投入したり、<br />テストや本番運用フェーズでデータの一部の値を変更したいときに使用する
- superuser（システム管理者）と特別な権限を与えられたユーザーのみが利用する想定になっている
  - 専用のログイン画面が用意されている
  - superuser：`is_superuser`属性がTrueになっているUserのこと
    - 管理サイトで登録されたモデルのレコードを自由に追加・変更・削除することができる
- 管理サイトのログイン画面のURLは、プロジェクトの初期設定では`/admin/`になっている
  - `urls.py`で変更することが可能
  
*プロジェクト初回作成時の`urls.py`*
~~~python
from django.urls import path
from django.contrib import admin

urlpatterns = [
    path(r'^admin/', admin.site.urls)
]
~~~

#### 管理サイトの準備
1. モデルクラスの準備
1. `django.contrib.admin.site`という`AdminSite`クラスのグローバルオブジェクトにモデルを登録(register)する
  - この処理は各アプリケーションの`admin.py`モジュールに書くのが通例

## 13.2 モデルの登録方法

### 作成されたモデルを管理サイトに登録する
- 各アプリケーションディレクトリに配置された`admin.py`を編集する

#### 例：`shop`アプリケーションの「出版社（Publisher）」「著者（Auther）」「本（Book）」モデルを登録する
*shop/admin.py*
~~~python
from django.contrib import admin
from .models import Publisher, Author, Book

admin.site.register(Publisher)
admin.site.register(Author)
admin.site.register(Book)
~~~

## 13.3 一部機能のカスタマイズ方法
- `ModelAdmin`という管理サイトのオプションを管理しているクラスを使うことで、管理サイトの一部の機能をカスタマイズすることができる

### 例：`ModelAdmin`を使って管理サイトの表示フィールドを変更
- 一覧画面で出力されるフィールドを「タイトル」「出版社」「価格」に変更
- 一覧画面でのソート順を「価格」の高い順に変更
- 編集画面に表示するフィールドを「タイトル」「出版社」「著者」「価格」のみに変更

*shop/admin.py*
~~~python
from django.contrib import admin
from django import forms

from .models import Author, Book, Publisher

class BookModelAdmin(admin.ModelAdmin):
    # 一覧表示のフィールドを変更
    list_display = ('title', 'publisher', 'price')
    # 一覧表示のソート順を変更
    ordering = ('-price')
    # 編集画面のフィールドを変更
    fields = ('title', 'publisher', 'authors', 'price',)
    
admin.site.register(Publisher)
admin.site.register(Author)
admin.site.register(Book, BookModelAdmin)
~~~


### 例：管理サイトに`ModelForm`をによるバリデーションを追加

- レコードの保存時にタイトルに「Django」が含まれていなければエラーを表示する

*shop/admin.py*

~~~python
from django.contrib import admin
from django import forms

from .models import Author, Book, Publisher

class BookAdminForm(forms.ModelForm):
    def clean_title(self):
        value = self.cleaned_data['title']
        if 'Django' no in value:
            raise forms.ValidationError("タイトルには「Django」という文字を含めてください")
        return value
    
@admin.register(Book)
class BookModelAdmin(admin.ModelAdmin):
    list_display = ('title', 'publisher', 'price')
    ordering = ('-price')
    fields = ('title', 'publisher', 'authors', 'price',)
    form = BookAdminForm
    
admin.site.register(Publisher)
adimn.site.register(Author)
~~~

## 13.4 利用条件

### 管理サイトを有効化する条件

- 設定ファイルの`INSTALLED_APPS`に`django.contrib.admin`を追加
- 同じく`INSTALLED_APPS`に以下を追加
  - `django.contrib.auth`
  - `django.contrib.contenttypes`
  - `django.contrib.messages`
  - `django.contrib.sessions`
- 同じく`TEMPLATES.OPTIONS.context_processors`に以下を追加
  - `django.contrib.auth.context_processors.auth`
  - `django.contrib.messages.context_processors.messages`
- 同じく`MIDDLEWARE`に以下を追加
  - `django.contrib.auth.middleware.AuthenticationMiddleware`
  - `django.contrib.messages.middleware.MessageMiddleware`
- `URLconf`に管理サイトのURL設定を追加

***プロジェクト初回作成時のデフォルト設定のままで、全ての条件が満たされているので、各アプリケーションディレクトリの`admin.py`にモデルの登録処理を書くだけで良い***

### 管理サイトを利用できるユーザーの権限
- `is_stuff`と`is_active`属性の値がTrueである
  - 各モデルの「追加」「変更」「削除」のアクションを実行するためには、モデルごと・アクションごとの「ユーザーパーミッション」と呼ばれる権限が必要
    - 例：shopアプリケーションのBookモデルのレコードを追加する権限をユーザーに付与する<br />`shop | book | Can add book`
- `is_superuser`属性の値がTrueである
  - 全てのユーザーパーミッションが付与されているとみなされる

## 13.5 使い方
### 事前に完了しておく作業
- 管理サイトの有効化
- データベースのマイグレーション
- superuser（システム管理者）の作成
  - `createsuperuser`コマンドで作成する
    - `is_superuser`、`is_stuff`、`is_active`がいずれもTrueのユーザーが作成される
- Webサーバの起動

## 13.6 まとめ
- 「管理サイト」と呼ばれる、開発者やシステム管理者向けのマスタメンテ機能が付属されている
- 画面を使って、管理サイトに登録されたモデルのレコードを追加・変更・削除できる
- 管理サイトを利用するには特別な権限が必要
- 管理サイトに関わるコードは`admin.py`に書く